In [ ]:
!python /home/lucky/obj_det/vjt/data_preprocessing.py \
  --annotations /home/lucky/obj_det/vjt/annotations/instances_val2017.json \
  --images /home/lucky/obj_det/vjt/dataset/val2017 \
  --output /home/lucky/obj_det/vjt/dataset/data \
  --max-images 5000

loading annotations into memory...
Done (t=0.39s)
creating index...
index created!
Processing images: 100%|███████████████████| 5000/5000 [00:18<00:00, 268.75it/s]


In [ ]:
!wandb login 1adedc7f69090be4cfd442f650c4daa364c1d0b5

In [ ]:
!python /home/lucky/obj_det/vjt/train.py \
  --data_path /home/lucky/obj_det/vjt/dataset \
  --epochs 100 \
  --lr 0.0001 \
  --num_classes 256 \
  --batch_size 16 \
  --weight_decay 0.01 \
  --img_size 512,512

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: jhariaprabhat19 (jhariaprabhat19-indian-institute-of-science) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.19.9
wandb: Run data is saved locally in /home/yuvraj/obj_det/vjt/wandb/run-20250416_015203-9jced4by
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run zany-feather-18
wandb: ⭐️ View project at https://wandb.ai/jhariaprabhat19-indian-institute-of-science/segmentation-training
wandb: 🚀 View run at https://wandb.ai/jhariaprabhat19-indian-institute-of-science/segmentation-training/runs/9jced4by
/home/yuvraj/miniconda3/envs/seg-env/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:397: There is

In [ ]:
python /home/lucky/obj_det/vjt/train.py \
  --data_path /home/lucky/obj_det/vjt/dataset \
  --epochs 100 \
  --lr 0.00001 \
  --num_classes 91 \
  --batch_size 16 \
  --weight_decay 0.01 \
  --img_size 512,512


In [ ]:
import numpy as np
import cv2

mask = cv2.imread(str("/home/lucky/obj_det/vjt/dataset/masks/000000002532.png"), cv2.IMREAD_GRAYSCALE)

print(f"Unique mask values: {len(np.unique(mask))}")
print(f"Unique mask values: {np.unique(mask)}")


Unique mask values: 3
Unique mask values: [ 0  1 35]


In [ ]:
import segmentation_models_pytorch as smp
from train import SegmentationModel 

config = {
    "num_classes": 91,
    "class_weights": [1.0] * 91,
    "lr": 1e-4,
    "weight_decay": 0.01,
    "img_size": (512, 512)
}

# Load the model
checkpoint_path = "/home/lucky/LuckyML/checkpoints/epoch=53-step=16902.ckpt"  # updated path with 'lucky'
model = SegmentationModel.load_from_checkpoint(checkpoint_path, config=config)


In [10]:
model.eval()
model.cuda()  # if using GPU

SegmentationModel(
  (model): DeepLabV3Plus(
    (encoder): MixVisionTransformerEncoder(
      (patch_embed1): OverlapPatchEmbed(
        (proj): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4), padding=(3, 3))
        (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      )
      (patch_embed2): OverlapPatchEmbed(
        (proj): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      )
      (patch_embed3): OverlapPatchEmbed(
        (proj): Conv2d(128, 320, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (norm): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
      )
      (patch_embed4): OverlapPatchEmbed(
        (proj): Conv2d(320, 512, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2), dilation=(2, 2))
        (norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      )
      (block1): Sequential(
        (0): Block(
          (norm1): LayerNorm((64,), eps

In [ ]:
from torchvision.transforms import ToTensor
import cv2
from albumentations.pytorch import ToTensorV2
import albumentations as A
import torch

# Load an image
img = cv2.imread("/home/lucky/LuckyML/vjproject/vj_project/data/val2017/000000001532.jpg")
mask = cv2.imread(str("/home/lucky/LuckyML/vjproject/vj_project/data/masks/000000001532.png"), cv2.IMREAD_GRAYSCALE)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Resize and normalize (same as validation transform)
val_transform = A.Compose([
    A.Resize(512, 512),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2()
])
transformed = val_transform(image=img)
input_tensor = transformed["image"].unsqueeze(0).cuda()  # Add batch dimension

# Predict
with torch.no_grad():
    output = model(input_tensor)
    prediction = torch.argmax(output, dim=1).squeeze(0).cpu().numpy()
    print(prediction)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
